In [30]:
//%cflags:-lm

#include <stdio.h>
#include <stdint.h>
#include <math.h>

#define MAP_DIV_CONST   50000

#define MAP_W       64
#define MAP_H       64

#define GNSS_MUL				10000000l

#define min(a,b)                ((a)<(b)?(a):(b))
#define max(a,b)                ((a)>(b)?(a):(b))
#define abs(x)                  ((x)>0?(x):-(x))

static float lat_mult[] = {
	1.00001269263410,
	1.00016503034732,
	1.00062227564598,
	1.00138512587137,
	1.00245474614468,
	1.00383277371586,
	1.00552132409524,
	1.00752299900955,
	1.00984089623881,
	1.01247862140355,
	1.01544030178698,
	1.01873060229028,
	1.02235474363937,
	1.02631852297526,
	1.03062833698348,
	1.03529120773669,
	1.04031481145009,
	1.04570751037551,
	1.05147838808628,
	1.05763728844289,
	1.06419485855898,
	1.07116259613477,
	1.07855290156473,
	1.08637913528301,
	1.09465568086611,
	1.10339801447867,
	1.11262278132588,
	1.12234787986046,
	1.13259255459192,
	1.14337749845791,
	1.15472496584871,
	1.16665889752463,
	1.17920505883836,
	1.19239119287568,
	1.20624719035817,
	1.22080527842164,
	1.23610023069989,
	1.25216960150919,
	1.26905398736196,
	1.28679731954458,
	1.30544719209465,
	1.32505523022013,
	1.34567750504505,
	1.36737500157145,
	1.39021414794172,
	1.41426741553005,
	1.43961400112117,
	1.46634060453179,
	1.49454231757694,
	1.52432364338688,
	1.55579966888401,
	1.58909741791146,
	1.62435741829468,
	1.66173552331931,
	1.70140503710806,
	1.74355920469978,
	1.78841414194345,
	1.83621229854950,
	1.88722657098226,
	1.94176521201939,
	2.00017772298141,
};

// number of elements inside lat_mult array:
#define LAT_MULT_NUM_ELEMENTS (sizeof(lat_mult)/sizeof(float))

int32_t align_to_cache_grid(int32_t lon, int32_t lat, uint16_t zoom, int32_t * c_lon, int32_t * c_lat)
{
    int32_t step_x;
    int32_t step_y;

    uint16_t zoom_p = pow(2, min(zoom, 9));

    step_x = (zoom_p * (int64_t)GNSS_MUL) / MAP_DIV_CONST;

    //get bbox
    uint32_t map_w = (MAP_W * step_x);
    uint32_t map_h = 0;

    *c_lon = (lon / map_w) * map_w + map_w / 2;

    int32_t t_lat = 0;
    int8_t last_lat = -1;
    int32_t last_h = 0;

    while (t_lat + map_h < lat)
    {
        t_lat += map_h;

        if (t_lat / GNSS_MUL > last_lat)
        {
            last_lat = min(LAT_MULT_NUM_ELEMENTS - 1, abs(t_lat / GNSS_MUL));
            step_y = (zoom_p * (int64_t)GNSS_MUL / lat_mult[last_lat]) / MAP_DIV_CONST;
            last_h = map_h;
            map_h = MAP_H * step_y;
        }
    }

    *c_lat = t_lat + last_h / 2;
    
    return map_h;
}

int main()
{
    
    int32_t lon = 170393600;
    int32_t lat = 480066816;
    
//     for (uint8_t zoom = 0; zoom < 10; zoom++)
//     {
//         lon = 170393600;
//         while (lon < 176947200)
//         {
//             int32_t c_lon, c_lat;

//             int32_t map_w = align_to_cache_grid(lon, lat, zoom, &c_lon, &c_lat);

//             int32_t lon1 = c_lon - map_w / 2;
//             int32_t lon2 = c_lon + map_w / 2;

//             printf("%d %d %d %d\n", zoom, lon, lon1, lon2);
//             lon += 1000000;
//         }
//     }
    
    for (uint8_t zoom = 0; zoom < 5; zoom++)
    {
        lat = 450000000;
        while (lat < 450100000)
        {
            int32_t c_lon, c_lat;

            int32_t map_h = align_to_cache_grid(lon, lat, zoom, &c_lon, &c_lat);

            int32_t lat1 = c_lat + map_h / 2;
            int32_t lat2 = c_lat - map_h / 2;

            printf("%d %d ", lat1, lat2);
            lat += 5000;
        }
        printf("\n");
    }
    
}

450002016 449992864 450011008 450001984 450011008 450001984 450020032 450011008 450029056 450020032 450029056 450020032 450038080 450029056 450038080 450029056 450047104 450038080 450047104 450038080 450056128 450047104 450056128 450047104 450065152 450056128 450065152 450056128 450074176 450065152 450083200 450074176 450083200 450074176 450092224 450083200 450092224 450083200 450101248 450092224 
450009504 449991136 450009504 449991136 450027552 450009504 450027552 450009504 450027552 450009504 450027552 450009504 450045600 450027552 450045600 450027552 450045600 450027552 450045600 450027552 450063648 450045600 450063648 450045600 450063648 450045600 450081696 450063648 450081696 450063648 450081696 450063648 450081696 450063648 450099744 450081696 450099744 450081696 450099744 450081696 
450019456 449982656 450019456 449982656 450019456 449982656 450019456 449982656 450055616 450019456 450055616 450019456 450055616 450019456 450055616 450019456 450055616 450019456 450055616 45001945

In [27]:
//%cflags:-lm

#include <stdio.h>
#include <stdint.h>
#include <math.h>

#define MAP_DIV_CONST   50000

#define MAP_W       64
#define MAP_H       64

#define GNSS_MUL				10000000l

#define min(a,b)                ((a)<(b)?(a):(b))
#define max(a,b)                ((a)>(b)?(a):(b))
#define abs(x)                  ((x)>0?(x):-(x))

static float lat_mult[] = {
	1.00001269263410,
	1.00016503034732,
	1.00062227564598,
	1.00138512587137,
	1.00245474614468,
	1.00383277371586,
	1.00552132409524,
	1.00752299900955,
	1.00984089623881,
	1.01247862140355,
	1.01544030178698,
	1.01873060229028,
	1.02235474363937,
	1.02631852297526,
	1.03062833698348,
	1.03529120773669,
	1.04031481145009,
	1.04570751037551,
	1.05147838808628,
	1.05763728844289,
	1.06419485855898,
	1.07116259613477,
	1.07855290156473,
	1.08637913528301,
	1.09465568086611,
	1.10339801447867,
	1.11262278132588,
	1.12234787986046,
	1.13259255459192,
	1.14337749845791,
	1.15472496584871,
	1.16665889752463,
	1.17920505883836,
	1.19239119287568,
	1.20624719035817,
	1.22080527842164,
	1.23610023069989,
	1.25216960150919,
	1.26905398736196,
	1.28679731954458,
	1.30544719209465,
	1.32505523022013,
	1.34567750504505,
	1.36737500157145,
	1.39021414794172,
	1.41426741553005,
	1.43961400112117,
	1.46634060453179,
	1.49454231757694,
	1.52432364338688,
	1.55579966888401,
	1.58909741791146,
	1.62435741829468,
	1.66173552331931,
	1.70140503710806,
	1.74355920469978,
	1.78841414194345,
	1.83621229854950,
	1.88722657098226,
	1.94176521201939,
	2.00017772298141,
};

// number of elements inside lat_mult array:
#define LAT_MULT_NUM_ELEMENTS (sizeof(lat_mult)/sizeof(float))

int32_t align_to_cache_grid(int32_t lon, int32_t lat, uint16_t zoom, int32_t * c_lon, int32_t * c_lat)
{
    int32_t step_x;
    int32_t step_y;

    uint16_t zoom_p = pow(2, min(zoom, 9));

    step_x = (zoom_p * (int64_t)GNSS_MUL) / MAP_DIV_CONST;

    //get bbox
    uint32_t map_w = (MAP_W * step_x);
    uint32_t map_h = 0;

    *c_lon = (lon / map_w) * map_w + map_w / 2;

    int32_t t_lat = 0;
    int8_t last_lat = -1;

    while (t_lat + map_h < lat)
    {
        t_lat += map_h;

        if (t_lat / GNSS_MUL > last_lat)
        {
            last_lat = min(LAT_MULT_NUM_ELEMENTS - 1, abs(t_lat / GNSS_MUL));
            step_y = (zoom_p * (int64_t)GNSS_MUL / lat_mult[last_lat]) / MAP_DIV_CONST;
            map_h = MAP_H * step_y;
        }
    }

    *c_lat = t_lat + map_h / 2;
    
    return map_w;
}

int main()
{
    
    int32_t lon = 170393600;
    int32_t lat = 480066816;
    
    for (uint8_t zoom = 0; zoom < 5; zoom++)
    {
        lon = 450000000;
        while (lon < 450100000)
        {
            int32_t c_lon, c_lat;

            int32_t map_w = align_to_cache_grid(lon, lat, zoom, &c_lon, &c_lat);

            int32_t lon1 = c_lon - map_w / 2;
            int32_t lon2 = c_lon + map_w / 2;

            printf("%d %d ",lon1, lon2);
            lon += 5000;
        }
        printf("\n");
    }
    
}

449996800 450009600 449996800 450009600 450009600 450022400 450009600 450022400 450009600 450022400 450022400 450035200 450022400 450035200 450022400 450035200 450035200 450048000 450035200 450048000 450048000 450060800 450048000 450060800 450048000 450060800 450060800 450073600 450060800 450073600 450073600 450086400 450073600 450086400 450073600 450086400 450086400 450099200 450086400 450099200 
449996800 450022400 449996800 450022400 449996800 450022400 449996800 450022400 449996800 450022400 450022400 450048000 450022400 450048000 450022400 450048000 450022400 450048000 450022400 450048000 450048000 450073600 450048000 450073600 450048000 450073600 450048000 450073600 450048000 450073600 450073600 450099200 450073600 450099200 450073600 450099200 450073600 450099200 450073600 450099200 
449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 450048000 449996800 45004800